In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as K


img_width, img_height = 224, 224
train_data_dir = '../data/train/'
validation_data_dir = '../data/validation/'
nb_train_samples = 1350
nb_validation_samples = 150
epochs = 20
batch_size = 30


gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
from tensorflow.keras.layers import Input, DepthwiseConv2D, \
    Conv2D ,BatchNormalization, ReLU, AvgPool2D, Flatten, Dense

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

from tensorflow.keras.layers import Input, Conv2D, Concatenate, \
     MaxPool2D, GlobalAvgPool2D, Activation
 
 
def fire_block(x, squeeze_filters, expand_filters):
    squeezed = Conv2D(filters=squeeze_filters,
                      kernel_size=1,
                      activation='relu')(x)
    expanded_1x1 = Conv2D(filters=expand_filters,
                        kernel_size=1,
                        activation='relu')(squeezed)
    expanded_3x3 = Conv2D(filters=expand_filters,
                        kernel_size=3,
                        padding='same',
                        activation='relu')(squeezed)
 
    output = Concatenate()([expanded_1x1, expanded_3x3])
    return output
 
 
input = Input([224, 224, 3])
 
x = Conv2D(96, 7, strides=2, padding='same', activation='relu')(input)
x = MaxPool2D(3, strides=2, padding='same')(x)
 
 
x = fire_block(x, squeeze_filters=16, expand_filters=64)
x = fire_block(x, squeeze_filters=16, expand_filters=64)
x = fire_block(x, squeeze_filters=32, expand_filters=128)
x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)
 
x = fire_block(x, squeeze_filters=32, expand_filters=128)
x = fire_block(x, squeeze_filters=48, expand_filters=192)
x = fire_block(x, squeeze_filters=48, expand_filters=192)
x = fire_block(x, squeeze_filters=64, expand_filters=256)
x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)
 
x = fire_block(x, squeeze_filters=64, expand_filters=256)
 
 
x = Conv2D(filters=3, kernel_size=1)(x)
x = GlobalAvgPool2D()(x)
 
output = Activation('softmax')(x)
 
from tensorflow.keras import Model
model = Model(input, output)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_26 (Conv2D)              (None, 112, 112, 96) 14208       input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_3 (MaxPooling2D)  (None, 56, 56, 96)   0           conv2d_26[0][0]                  
__________________________________________________________________________________________________
conv2d_27 (Conv2D)              (None, 56, 56, 16)   1552        max_pooling2d_3[0][0]            
____________________________________________________________________________________________

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(lr=1e-3, momentum=0.9),
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Found 1350 images belonging to 3 classes.
Found 150 images belonging to 3 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
45/45 [==============================] - 13s 282ms/step - loss: 1.1051 - accuracy: 0.3556 - val_loss: 1.1008 - val_accuracy: 0.3333
Epoch 2/20
45/45 [==============================] - 13s 285ms/step - loss: 1.1004 - accuracy: 0.3148 - val_loss: 1.0972 - val_accuracy: 0.3400
Epoch 3/20
45/45 [==============================] - 12s 276ms/step - loss: 1.0997 - accuracy: 0.3237 - val_loss: 1.0985 - val_accuracy: 0.3533
Epoch 4/20
45/45 [==============================] - 13s 286ms/step - loss: 1.1004 - accuracy: 0.3200 - val_loss: 1.1005 - val_accuracy: 0.3067
Epoch 5/20
45/45 [==============================] - 13s 288ms/step - loss: 1.0994 - accuracy: 0.3274 - val_loss: 1.0978 - val_accuracy: 0.3533
Epoch 6/20
45/45 [==============================] - 13s 278ms/step - loss: 1.0997 - accuracy: 0.3311 - val_loss: 1.0987 - val_

In [4]:
model.save("./squeezenet_1.h5")